In [1]:
import numpy as np
import mne
from mne.decoding import CSP
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.neural_network import MLPClassifier  # Neural Network Classifier
from sklearn.model_selection import StratifiedKFold, cross_val_score, cross_val_predict, GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from mne.filter import filter_data
import matplotlib.pyplot as plt
# Import the required Python libraries
import os
import re
from pathlib import Path
import pandas as pd
import numpy as np
import mne
from mne import Epochs

In [2]:
# Define the path to your raw_data directory
data_dir = Path('../data/raw data')  # Adjust this path as needed

# Verify the directory exists
if not data_dir.exists():
    raise FileNotFoundError(f"The specified data directory does not exist: {data_dir}")

# Define the specific runs to process, including both two-digit and three-digit run numbers
desired_runs = ['04', '08', '12']

# Compile a regular expression pattern to match filenames like S001R04.edf, S002R08.edf, S001R012.edf, etc.
# This pattern ensures that only R04, R08, R12 are matched
edf_pattern = re.compile(r'^S\d{3}R0?(04|08|12)\.edf$', re.IGNORECASE)

In [3]:
# Initialize a list to store file information
file_info = []

# Counter for processed subject directories
processed_subjects = 0

# Iterate through each subject folder in raw_data
for subject_dir in data_dir.iterdir():
    if subject_dir.is_dir() and re.match(r'^S\d{3}$', subject_dir.name, re.IGNORECASE):
        processed_subjects += 1
        # Iterate through each file in the subject directory
        for file in subject_dir.iterdir():
            if file.is_file() and edf_pattern.match(file.name):
                # Derive the corresponding event file name
                base_name = file.stem  # e.g., S001R04
                event_file = f"{base_name}.edf.event"
                event_path = subject_dir / event_file
                
                # Check if the event file exists
                event_exists = event_path.exists()
                
                # Append the information to the list
                file_info.append({
                    'subject': subject_dir.name,
                    'edf_file': file.name,
                    'event_file': event_file,
                    'event_exists': event_exists,
                    'edf_path': str(file.resolve()),
                    'event_path': str(event_path.resolve()) if event_exists else None
                })

# Convert the list to a DataFrame for better visualization
df_files = pd.DataFrame(file_info)

# Display summary of processed subject directories
print(f"Total subject directories processed: {processed_subjects}")

Total subject directories processed: 109


In [ ]:
# Total number of matched .edf files
total_edf = df_files.shape[0]

# Number of event files present
total_events = df_files['event_exists'].sum()

# Number of missing event files
missing_events = total_edf - total_events

print(f"Total matched .edf files: {total_edf}")
print(f"Number of corresponding event files found: {total_events}")
print(f"Number of missing event files: {missing_events}")

In [4]:
# Filter the DataFrame for missing event files
df_missing_events = df_files[~df_files['event_exists']]

# Display the list of .edf files without corresponding event files
if not df_missing_events.empty:
    print("\nFiles missing corresponding event files:")
    display(df_missing_events[['subject', 'edf_file', 'event_file']])
else:
    print("\nAll matched .edf files have corresponding event files.")


All matched .edf files have corresponding event files.


In [5]:
# Display the entire DataFrame of matched files
print("List of All Matched .edf Files:")
display(df_files[['subject', 'edf_file', 'event_file', 'event_exists']])

List of All Matched .edf Files:


,subject,edf_file,event_file,event_exists
0,S038,S038R12.edf,S038R12.edf.event,True
1,S038,S038R04.edf,S038R04.edf.event,True
2,S038,S038R08.edf,S038R08.edf.event,True
3,S007,S007R08.edf,S007R08.edf.event,True
4,S007,S007R04.edf,S007R04.edf.event,True
...,...,...,...,...
322,S106,S106R04.edf,S106R04.edf.event,True
323,S106,S106R08.edf,S106R08.edf.event,True
324,S108,S108R04.edf,S108R04.edf.event,True
325,S108,S108R12.edf,S108R12.edf.event,True


In [6]:
# Initialize lists to collect data and labels from all files
X_all = []
y_all = []


In [7]:
# Define desired event types
desired_event_types = ['Rest', 'Imagining Left Hand', 'Imagining Right Hand']

# Iterate through each matched EDF file
for idx, row in df_files.iterrows():
    edf_path = row['edf_path']
    event_exists = row['event_exists']
    subject = row['subject']
    edf_file = row['edf_file']
    
    print(f"\nProcessing file: {edf_file} for subject: {subject}")
    
    if not event_exists:
        print(f"Skipping {edf_file} as it lacks an event file.")
        continue
    
    try:
        # Load the raw EEG data

        raw = mne.io.read_raw_edf(edf_path, preload=True, verbose=False)
        # Convert annotations to events and event_id
        events, event_id = mne.events_from_annotations(raw) # This line is missing in your code

        # Define event dictionary
        event_dict = {
            'T0': 1,                # Resting State
            'T1': 2,  # 'Imagining Left Hand': 
            'T2': 3  # 'Imagining Right Hand':
        }



        # Update event_id with descriptive labels
        event_id = {
            'T0': 1,
            'T1': 2,
            'T2': 3
        }

        
        # Access annotations
        annotations = raw.annotations
        
        # Define epoch parameters
        tmin = -0.2  # 200 ms before the event
        tmax = 3.8   # 3800 ms after the event
        baseline = (None, 0)  # Baseline correction
        
        # Convert annotations to events
        events, _ = mne.events_from_annotations(raw, event_id=event_dict)
        
        if events.size == 0:
            print(f"No events found in {edf_file}. Skipping.")
            continue
        
        # Create epochs
        epochs = Epochs(raw, events, event_id=event_dict, tmin=tmin, tmax=tmax, baseline=baseline, preload=True, verbose=False)
        
        # Select only 'Imagining Left Hand' and 'Imagining Right Hand' epochs
        motor_epochs = epochs[['T1', 'T2']]
        
        if len(motor_epochs) == 0:
            print(f"No motor-related epochs found in {edf_file}. Skipping.")
            continue
        
        # Extract labels: 0 for Left, 1 for Right
        labels = motor_epochs.events[:, -1] - event_dict['T1']
        
        # Get the data as a NumPy array: shape (n_epochs, n_channels, n_times)
        X = motor_epochs.get_data()
        y = labels
        
        # Append to the aggregate lists
        X_all.append(X)
        y_all.append(y)
        
        print(f"Loaded {X.shape[0]} epochs from {edf_file}.")
        
    except Exception as e:
        print(f"Failed to process {edf_file}. Error: {e}")


'''
# Define the path to the EDF file
edf_file = '../data/raw data/S001/S001R04.edf'

# Load the raw EEG data
raw = mne.io.read_raw_edf(edf_file, preload=True)

# Display basic information about the loaded data
print(raw.info)

# Plot raw EEG data interactively (optional)
# raw.plot(duration=30, n_channels=30, scalings='auto', title='Raw EEG Signals', show=True)

# Load the raw EEG data with annotations
raw = mne.io.read_raw_edf(edf_file, preload=True)

# Access annotations
annotations = raw.annotations

# Display annotations
print(annotations)

# Define event dictionary
event_dict = {
    'Rest': 1,                # T0
    'Imagining Left Hand': 2,  # T1
    'Imagining Right Hand': 3  # T2
}

# Convert annotations to events and event_id
events, event_id = mne.events_from_annotations(raw)

# Update event_id with descriptive labels
event_id = {
    'Rest': 1,
    'Imagining Left Hand': 2,
    'Imagining Right Hand': 3
}

print(event_id)

from mne import Epochs

# Define epoch parameters
tmin = -0.2  # 200 ms before the event
tmax = 3.8   # 3800 ms after the event
baseline = (None, 0)  # Baseline correction

# Create epochs
epochs = Epochs(raw, events, event_id=event_id, tmin=tmin, tmax=tmax, baseline=baseline, preload=True)

# Display epoch information
print(epochs)

# Plot an average epoch for 'Imagining Left Hand' (optional)
# fig = epochs['Imagining Left Hand'].average().plot()
# fig.suptitle('Imagining Left Hand')
# plt.show()

# Plot an average epoch for 'Imagining Right Hand' (optional)
# fig = epochs['Imagining Right Hand'].average().plot()
# fig.suptitle('Imagining Right Hand')
# plt.show()

# Plot an average epoch for 'Rest' (optional)
# fig = epochs['Rest'].average().plot()
# fig.suptitle('Rest')
# plt.show()

# Select only 'Imagining Left Hand' and 'Imagining Right Hand' epochs
motor_epochs = epochs[['Imagining Left Hand', 'Imagining Right Hand']]

# Extract labels: 0 for Left, 1 for Right
labels = motor_epochs.events[:, -1] - 2  # 'Imagining Left Hand' = 2, 'Imagining Right Hand' = 3

# Verify the distribution of classes
print("Class distribution:", np.bincount(labels))

# Get the data as a NumPy array: shape (n_epochs, n_channels, n_times)
X = motor_epochs.get_data()
y = labels

print(f"Shape of X: {X.shape}")
print(f"Shape of y: {y.shape}")
'''


Processing file: S038R12.edf for subject: S038
Used Annotations descriptions: [np.str_('T0'), np.str_('T1'), np.str_('T2')]
Used Annotations descriptions: [np.str_('T0'), np.str_('T1'), np.str_('T2')]
Loaded 15 epochs from S038R12.edf.

Processing file: S038R04.edf for subject: S038
Used Annotations descriptions: [np.str_('T0'), np.str_('T1'), np.str_('T2')]
Used Annotations descriptions: [np.str_('T0'), np.str_('T1'), np.str_('T2')]
Loaded 15 epochs from S038R04.edf.

Processing file: S038R08.edf for subject: S038
Used Annotations descriptions: [np.str_('T0'), np.str_('T1'), np.str_('T2')]
Used Annotations descriptions: [np.str_('T0'), np.str_('T1'), np.str_('T2')]
Loaded 15 epochs from S038R08.edf.

Processing file: S007R08.edf for subject: S007
Used Annotations descriptions: [np.str_('T0'), np.str_('T1'), np.str_('T2')]
Used Annotations descriptions: [np.str_('T0'), np.str_('T1'), np.str_('T2')]
Loaded 15 epochs from S007R08.edf.

Processing file: S007R04.edf for subject: S007
Used

/var/folders/64/t0s5x1395f38bgfnfwy0_cdh0000gn/T/ipykernel_19654/1908462150.py:30: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw = mne.io.read_raw_edf(edf_path, preload=True, verbose=False)
/var/folders/64/t0s5x1395f38bgfnfwy0_cdh0000gn/T/ipykernel_19654/1908462150.py:30: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw = mne.io.read_raw_edf(edf_path, preload=True, verbose=False)
/var/folders/64/t0s5x1395f38bgfnfwy0_cdh0000gn/T/ipykernel_19654/1908462150.py:30: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw = mne.io.read_raw_edf(edf_path, preload=True, verbose=False)


Loaded 15 epochs from S106R04.edf.

Processing file: S106R08.edf for subject: S106
Used Annotations descriptions: [np.str_('T0'), np.str_('T1'), np.str_('T2')]
Used Annotations descriptions: [np.str_('T0'), np.str_('T1'), np.str_('T2')]
Loaded 15 epochs from S106R08.edf.

Processing file: S108R04.edf for subject: S108
Used Annotations descriptions: [np.str_('T0'), np.str_('T1'), np.str_('T2')]
Used Annotations descriptions: [np.str_('T0'), np.str_('T1'), np.str_('T2')]
Loaded 15 epochs from S108R04.edf.

Processing file: S108R12.edf for subject: S108
Used Annotations descriptions: [np.str_('T0'), np.str_('T1'), np.str_('T2')]
Used Annotations descriptions: [np.str_('T0'), np.str_('T1'), np.str_('T2')]
Loaded 15 epochs from S108R12.edf.

Processing file: S108R08.edf for subject: S108
Used Annotations descriptions: [np.str_('T0'), np.str_('T1'), np.str_('T2')]
Used Annotations descriptions: [np.str_('T0'), np.str_('T1'), np.str_('T2')]
Loaded 15 epochs from S108R08.edf.


'\n# Define the path to the EDF file\nedf_file = \'../data/raw data/S001/S001R04.edf\'\n\n# Load the raw EEG data\nraw = mne.io.read_raw_edf(edf_file, preload=True)\n\n# Display basic information about the loaded data\nprint(raw.info)\n\n# Plot raw EEG data interactively (optional)\n# raw.plot(duration=30, n_channels=30, scalings=\'auto\', title=\'Raw EEG Signals\', show=True)\n\n# Load the raw EEG data with annotations\nraw = mne.io.read_raw_edf(edf_file, preload=True)\n\n# Access annotations\nannotations = raw.annotations\n\n# Display annotations\nprint(annotations)\n\n# Define event dictionary\nevent_dict = {\n    \'Rest\': 1,                # T0\n    \'Imagining Left Hand\': 2,  # T1\n    \'Imagining Right Hand\': 3  # T2\n}\n\n# Convert annotations to events and event_id\nevents, event_id = mne.events_from_annotations(raw)\n\n# Update event_id with descriptive labels\nevent_id = {\n    \'Rest\': 1,\n    \'Imagining Left Hand\': 2,\n    \'Imagining Right Hand\': 3\n}\n\nprint(event

In [8]:
print(f"Total number of files processed: {len(X_all)}")
print(f"Total number of labels collected: {len(y_all)}")

Total number of files processed: 327
Total number of labels collected: 327


In [13]:
all_epochs = []
all_labels = []

for idx, row in df_files.iterrows():
    # ... load and process each file to get X and y ...
    # Suppose after processing a file, you have:
    # X (n_epochs, n_channels, n_times)
    # y (n_epochs,)

    all_epochs.append(X)
    all_labels.append(y)

X_all = np.concatenate(all_epochs, axis=0)
y_all = np.concatenate(all_labels, axis=0)

print("X_all shape:", X_all.shape)
print("y_all shape:", y_all.shape)

X_all shape: (4905, 64, 641)
y_all shape: (4905,)


In [17]:
import numpy as np
import logging
from sklearn.base import BaseEstimator, TransformerMixin
from mne.filter import filter_data
from mne.decoding import CSP

# Configure logging: 
# Set level=logging.INFO for less verbosity, level=logging.DEBUG for more details
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

class FBCSP(BaseEstimator, TransformerMixin):
    def __init__(self, freq_bands, n_csp=4, sfreq=160, filter_method='iir', csp_reg=None):
        """
        Filter Bank Common Spatial Pattern (FBCSP) transformer.

        Parameters
        ----------
        freq_bands : list of tuple
            List of frequency bands (l_freq, h_freq).
        n_csp : int, optional
            Number of CSP components per band, by default 4.
        sfreq : float, optional
            Sampling frequency of the EEG data, by default 160.
        filter_method : str, optional
            Filtering method to use ('iir' or 'fir'), by default 'iir'.
        csp_reg : None or str or float, optional
            Regularization parameter for CSP. 
            Examples: None, 'ledoit_wolf', 'oas', or float values. See MNE CSP docs.
        """
        self.freq_bands = freq_bands
        self.n_csp = n_csp
        self.sfreq = sfreq
        self.filter_method = filter_method
        self.csp_reg = csp_reg
        self.csp_list_ = []

    def fit(self, X, y):
        """
        Fit CSP models for each frequency band.

        Parameters
        ----------
        X : np.ndarray, shape (n_trials, n_channels, n_times)
            EEG data.
        y : np.ndarray, shape (n_trials,)
            Class labels.

        Returns
        -------
        self : object
            Fitted FBCSP instance.
        """
        # Validate input shapes
        if X.ndim != 3:
            raise ValueError(f"X should be 3D (n_trials, n_channels, n_times). Got shape: {X.shape}")
        if X.shape[0] != len(y):
            raise ValueError("Number of trials in X does not match length of y.")

        self.csp_list_ = []
        for band in self.freq_bands:
            l_freq, h_freq = band
            logging.info(f"Fitting CSP for band: {l_freq}-{h_freq} Hz")

            # Band-pass filter the data for the current frequency band
            X_filtered = np.array([
                filter_data(trial, sfreq=self.sfreq, l_freq=l_freq, h_freq=h_freq, 
                            method=self.filter_method, verbose=False)
                for trial in X
            ])

            # Initialize and fit CSP for the current band
            csp = CSP(n_components=self.n_csp, reg=self.csp_reg, log=True, norm_trace=False)
            csp.fit(X_filtered, y)
            self.csp_list_.append(csp)

        return self

    def transform(self, X):
        """
        Transform EEG data using the fitted CSP models.

        Parameters
        ----------
        X : np.ndarray, shape (n_trials, n_channels, n_times)
            EEG data to transform.

        Returns
        -------
        X_features : np.ndarray, shape (n_trials, n_bands * n_csp)
            Extracted features.
        """
        # Validate input shapes
        if X.ndim != 3:
            raise ValueError(f"X should be 3D (n_trials, n_channels, n_times). Got shape: {X.shape}")

        # Ensure that the CSPs have been fitted
        if not self.csp_list_:
            raise RuntimeError("FBCSP has not been fitted yet. Call fit before transform.")

        features = []
        for idx, (l_freq, h_freq) in enumerate(self.freq_bands):
            logging.info(f"Transforming data for band: {l_freq}-{h_freq} Hz")
            csp = self.csp_list_[idx]

            # Band-pass filter the data for the current frequency band
            X_filtered = np.array([
                filter_data(trial, sfreq=self.sfreq, l_freq=l_freq, h_freq=h_freq, 
                            method=self.filter_method, verbose=False)
                for trial in X
            ])

            # Apply CSP transformation
            X_csp = csp.transform(X_filtered)

            # Extract log-variance features
            # shape: (n_trials,) -> reshape to (n_trials, 1)
            X_logvar = np.log(np.var(X_csp, axis=1)).reshape(-1, 1)

            features.append(X_logvar)

        # Concatenate features from all bands
        X_features = np.column_stack(features)
        return X_features


In [19]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.pipeline import Pipeline

#frequency bands (already defined previously)
freq_bands = [
    (8, 12),    # Alpha
    (13, 20),   # Beta 1
    (21, 30),   # Beta 2
    (31, 40),   # Low Gamma
    (41, 50),   # High Gamma
    (51, 60),   # Very High Gamma
    (61, 70),   # Ultra High Gamma
    (71, 79)    # Max Gamma
]

# Instantiate the FBCSP transformer with desired parameters
fbcsp = FBCSP(freq_bands=freq_bands, n_csp=4, sfreq=160, filter_method='iir', csp_reg=None)

# Instantiate the Random Forest classifier with 10 trees
rf = RandomForestClassifier(n_estimators=10, random_state=42)

# Build the pipeline: first apply FBCSP, then classify with Random Forest
pipeline = Pipeline([
    ('fbcsp', fbcsp),
    ('classifier', rf)
])

# Define cross-validation strategy
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Perform cross-validation
scores = cross_val_score(pipeline, X_all, y_all, cv=cv, scoring='accuracy')

print("Cross-validation accuracy scores:", scores)
print(f"Mean accuracy: {scores.mean() * 100:.2f}%")
print(f"Standard deviation: {scores.std() * 100:.2f}%")

# Fit the pipeline on all data
pipeline.fit(X_all, y_all)

2024-12-09 11:12:19,700 - INFO - Fitting CSP for band: 8-12 Hz


Computing rank from data with rank=None
    Using tolerance 0.0017 (2.2e-16 eps * 64 dim * 1.2e+11  max singular value)
    Estimated rank (data): 64
    data: rank 64 computed from 64 data channels with 0 projectors
Reducing data rank from 64 -> 64
Estimating class=0 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.


2024-12-09 11:13:10,872 - INFO - Fitting CSP for band: 13-20 Hz


Computing rank from data with rank=None
    Using tolerance 0.0013 (2.2e-16 eps * 64 dim * 8.9e+10  max singular value)
    Estimated rank (data): 64
    data: rank 64 computed from 64 data channels with 0 projectors
Reducing data rank from 64 -> 64
Estimating class=0 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.


2024-12-09 11:13:58,232 - INFO - Fitting CSP for band: 21-30 Hz


...

Computing rank from data with rank=None
    Using tolerance 0.00074 (2.2e-16 eps * 64 dim * 5.2e+10  max singular value)
    Estimated rank (data): 64
    data: rank 64 computed from 64 data channels with 0 projectors
Reducing data rank from 64 -> 64
Estimating class=0 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.


2024-12-09 12:22:07,848 - INFO - Fitting CSP for band: 71-79 Hz


Computing rank from data with rank=None
    Using tolerance 0.00061 (2.2e-16 eps * 64 dim * 4.3e+10  max singular value)
    Estimated rank (data): 64
    data: rank 64 computed from 64 data channels with 0 projectors
Reducing data rank from 64 -> 64
Estimating class=0 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.


2024-12-09 12:23:10,292 - INFO - Transforming data for band: 8-12 Hz
2024-12-09 12:23:56,566 - INFO - Transforming data for band: 13-20 Hz
2024-12-09 12:24:40,436 - INFO - Transforming data for band: 21-30 Hz
2024-12-09 12:25:27,017 - INFO - Transforming data for band: 31-40 Hz
2024-12-09 12:26:14,042 - INFO - Transforming data for band: 41-50 Hz
2024-12-09 12:27:01,528 - INFO - Transforming data for band: 51-60 Hz
2024-12-09 12:27:44,624 - INFO - Transforming data for band: 61-70 Hz
2024-12-09 12:28:29,435 - INFO - Transforming data for band: 71-79 Hz


Pipeline(steps=[('fbcsp',
                 FBCSP(freq_bands=[(8, 12), (13, 20), (21, 30), (31, 40),
                                   (41, 50), (51, 60), (61, 70), (71, 79)])),
                ('classifier',
                 RandomForestClassifier(n_estimators=10, random_state=42))])

In [21]:
# Check out the rf classifier's accuracy
y_pred = pipeline.predict(X_all)
accuracy = np.mean(y_pred == y_all)
print(f"Accuracy on the entire dataset: {accuracy * 100:.2f}%")


2024-12-09 12:38:19,194 - INFO - Transforming data for band: 8-12 Hz


KeyboardInterrupt: 

In [19]:
class FBCSP(BaseEstimator, TransformerMixin):
    def __init__(self, freq_bands, n_csp=4, sfreq=160):
        """
        Initialize the FBCSP transformer.
        
        Parameters:
        - freq_bands: List of tuples defining frequency bands.
        - n_csp: Number of CSP components per band.
        - sfreq: Sampling frequency of the EEG data.
        """
        self.freq_bands = freq_bands
        self.n_csp = n_csp
        self.sfreq = sfreq
        self.csp_list_ = []  # To store CSP models for each band

    def fit(self, X, y):
        """
        Fit CSP models for each frequency band.
        
        Parameters:
        - X: EEG data, shape (n_trials, n_channels, n_times)
        - y: Labels, shape (n_trials,)
        
        Returns:
        - self
        """
        self.csp_list_ = []  # Reset in case of multiple fits
        for band in self.freq_bands:
            print(f'Fitting CSP for band: {band[0]}-{band[1]} Hz')
            # Band-pass filter the data for the current frequency band
            X_filtered = np.array([
                filter_data(trial, sfreq=self.sfreq, l_freq=band[0], h_freq=band[1], method='iir', verbose=False)
                for trial in X
            ])
            
            # Initialize and fit CSP
            csp = CSP(n_components=self.n_csp, reg=None, log=True, norm_trace=False)
            csp.fit(X_filtered, y)
            self.csp_list_.append(csp)
        return self

    def transform(self, X):
        """
        Transform EEG data using the fitted CSP models.
        
        Parameters:
        - X: EEG data, shape (n_trials, n_channels, n_times)
        
        Returns:
        - X_features: Extracted features, shape (n_trials, n_bands * n_csp)
        """
        features = []
        for idx, band in enumerate(self.freq_bands):
            print(f'Transforming data for band: {band[0]}-{band[1]} Hz')
            csp = self.csp_list_[idx]
            # Band-pass filter the data for the current frequency band
            X_filtered = np.array([
                filter_data(trial, sfreq=self.sfreq, l_freq=band[0], h_freq=band[1], method='iir', verbose=False)
                for trial in X
            ])
            
            # Apply CSP transformation
            X_csp = csp.transform(X_filtered)
            
            # Extract log-variance features
            X_logvar = np.log(np.var(X_csp, axis=1))
            
            # Reshape X_logvar to (n_trials, 1) to make it 2D
            X_logvar = X_logvar.reshape(-1, 1)
            
            # Append the reshaped feature
            features.append(X_logvar)
        
        # Concatenate features from all bands along axis=1
       
        X_features = np.column_stack(features)
        return X_features

In [20]:
# Initialize FBCSP transformer
fbcsp = FBCSP(freq_bands=freq_bands, n_csp=4, sfreq=160)  # Replace sfreq with your actual sampling frequency

# Initialize Neural Network classifier
mlp = MLPClassifier(hidden_layer_sizes=(100,), max_iter=500, random_state=42)

# Define the classification pipeline
pipeline = Pipeline([
    ('fbcsp', fbcsp),
    ('classifier', mlp)
])

In [ ]:
# Define cross-validation strategy
n_splits = 5  # Adjust based on your dataset size
cv = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Perform cross-validation
print("Starting Cross-Validation...")
scores = cross_val_score(pipeline, X, y, cv=cv, scoring='accuracy')

print(f"\nCross-Validation Accuracy Scores: {scores}")
print(f"Mean Accuracy: {scores.mean() * 100:.2f}%")
print(f"Standard Deviation: {scores.std() * 100:.2f}%")